## Linear Classifier in TensorFlow 
Using Low Level API in Eager Execution mode

### Load tensorflow

In [1]:
%tensorflow_version 2.x
import tensorflow as tf

TensorFlow 2.x selected.


In [2]:
print(tf.__version__)

2.1.0-rc1


In [0]:
#Enable Eager Execution if using tensflow version < 2.0
#From tensorflow v2.0 onwards, Eager Execution will be enabled by default


### Collect Data

In [3]:
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [0]:
import pandas as pd

In [0]:
data = pd.read_csv('/gdrive/My Drive/ClassNotes/R6_Ext/prices.csv')

### Check all columns in the dataset

In [6]:
data.columns

Index(['date', 'symbol', 'open', 'close', 'low', 'high', 'volume'], dtype='object')

In [7]:
data.info()   ## date and symbol are Object type rest are numeric

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 851264 entries, 0 to 851263
Data columns (total 7 columns):
date      851264 non-null object
symbol    851264 non-null object
open      851264 non-null float64
close     851264 non-null float64
low       851264 non-null float64
high      851264 non-null float64
volume    851264 non-null float64
dtypes: float64(5), object(2)
memory usage: 45.5+ MB


In [8]:
data.head()

,date,symbol,open,close,low,high,volume
0,2016-01-05 00:00:00,WLTW,123.430000,125.839996,122.309998,126.250000,2163600.0
1,2016-01-06 00:00:00,WLTW,125.239998,119.980003,119.940002,125.540001,2386400.0
2,2016-01-07 00:00:00,WLTW,116.379997,114.949997,114.930000,119.739998,2489500.0
3,2016-01-08 00:00:00,WLTW,115.480003,116.620003,113.500000,117.440002,2006300.0
4,2016-01-11 00:00:00,WLTW,117.010002,114.970001,114.089996,117.330002,1408600.0


In [9]:
data.describe()   ## data is skewed as there is big diff between mean and medians.

,open,close,low,high,volume
count,851264.000000,851264.000000,851264.000000,851264.000000,8.512640e+05
mean,70.836986,70.857109,70.118414,71.543476,5.415113e+06
std,83.695876,83.689686,82.877294,84.465504,1.249468e+07
min,0.850000,0.860000,0.830000,0.880000,0.000000e+00
25%,33.840000,33.849998,33.480000,34.189999,1.221500e+06
50%,52.770000,52.799999,52.230000,53.310001,2.476250e+06
75%,79.879997,79.889999,79.110001,80.610001,5.222500e+06
max,1584.439941,1578.130005,1549.939941,1600.930054,8.596434e+08


### Drop columns `date` and  `symbol`

In [0]:
data=data.drop(labels=["date","symbol" ],  axis=1)

In [11]:
data.tail()

,open,close,low,high,volume
851259,103.309998,103.199997,102.849998,103.930000,973800.0
851260,43.070000,43.040001,42.689999,43.310001,1938100.0
851261,53.639999,53.529999,53.270000,53.740002,1701200.0
851262,44.730000,45.450001,44.410000,45.590000,1380900.0
851263,54.200001,53.630001,53.389999,54.480000,705100.0


In [12]:
data.head()

,open,close,low,high,volume
0,123.430000,125.839996,122.309998,126.250000,2163600.0
1,125.239998,119.980003,119.940002,125.540001,2386400.0
2,116.379997,114.949997,114.930000,119.739998,2489500.0
3,115.480003,116.620003,113.500000,117.440002,2006300.0
4,117.010002,114.970001,114.089996,117.330002,1408600.0


### Consider only first 1000 rows in the dataset for building feature set and target set
Target 'Volume' has very high values. Divide 'Volume' by 1000,000

In [13]:
data_feature=data.loc[0:1000]
data_feature["volume"]=data_feature["volume"]/1000000

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [14]:
data_feature.dtypes

open      float64
close     float64
low       float64
high      float64
volume    float64
dtype: object

In [15]:
data_feature["volume"].max()   ## now the values has been reduced.

215.6202

### Divide the data into train and test sets

In [0]:
import numpy as np
X = np.array(data_feature.drop("volume", axis=1))
y = np.array(data_feature["volume"])

In [0]:
from sklearn.model_selection import train_test_split

In [0]:
test_size = 0.30 # taking 70:30 training and test set
seed = 7  # Random numbmer seeding for reapeatability of the code
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=seed)


#### Convert Training and Test Data to numpy float32 arrays

In [0]:
X_train=X_train.astype('float32')
X_test=X_test.astype('float32')

y_train=y_train.astype('float32')
y_test=y_test.astype('float32')

### Normalize the data
You can use Normalizer from sklearn.preprocessing

In [0]:
from sklearn.preprocessing import Normalizer

norm=Normalizer()
X_train_norm=norm.fit_transform(X_train)
X_test_norm=norm.fit_transform(X_test)


In [21]:
X_test_norm  ## normalised here..

array([[0.49739146, 0.5033128 , 0.4944308 , 0.50479317],
       [0.5002627 , 0.5026732 , 0.49291062, 0.50407934],
       [0.50025874, 0.5013112 , 0.49651673, 0.50189584],
       ...,
       [0.4980231 , 0.5010118 , 0.49707213, 0.5038646 ],
       [0.5091813 , 0.5000049 , 0.47896615, 0.51119566],
       [0.49635795, 0.5001823 , 0.49635795, 0.50702584]], dtype=float32)

## Building the Model in tensorflow

1.Define Weights and Bias, use tf.zeros to initialize weights and Bias

In [22]:
X_train_norm.shape  ## W to have corresponding shape as of dimensions in Train data

(700, 4)

In [23]:
W = tf.zeros(shape=(4, 1))  ## W represents Weights
b = tf.zeros(shape=(1))  ## b represents Bias

b.dtype

tf.float32

2.Define a function to calculate prediction

In [0]:
@tf.function
def prediction(features, W, b):
    y_pred = tf.add(tf.matmul(features, W), b)  
    return y_pred

3.Loss (Cost) Function [Mean square error]

In [0]:
@tf.function
def loss(y_actual, y_predicted):
    diff = y_actual - y_predicted
    sqr = tf.square(diff)
    avg = tf.reduce_mean(sqr)
    return avg

4.Function to train the Model

1.   Record all the mathematical steps to calculate Loss
2.   Calculate Gradients of Loss w.r.t weights and bias
3.   Update Weights and Bias based on gradients and learning rate to minimize loss

In [0]:
@tf.function
def train(x, y_actual, w, b, learning_rate=0.01):
    
    # Record mathematical operations on 'tape' to calculate loss
    with tf.GradientTape() as t:
        t.watch([w,b])
        current_prediction = prediction(x, w, b)
        current_loss = loss(y_actual, current_prediction)
    
    # Calculate Gradients for Loss with respect to Weights and Bias
    dw, db = t.gradient(current_loss,[w, b])
    
    # Update Weights and Bias
    w = w - learning_rate * dw
    b = b - learning_rate * db
    
    return w, b

## Train the model for 100 epochs 
1. Observe the training loss at every iteration
2. Observe Train loss at every 5th iteration

In [27]:
for i in range(100):    ## 1.observing training loss at every step
    W, b = train(X_train_norm, y_train, W, b)
    print('Current Training Loss on iteration', i, loss(y_train, prediction(X_train_norm, W, b)).numpy())


Current Training Loss on iteration 0 240.44244
Current Training Loss on iteration 1 238.437
Current Training Loss on iteration 2 236.5888
Current Training Loss on iteration 3 234.88547
Current Training Loss on iteration 4 233.31569
Current Training Loss on iteration 5 231.86897
Current Training Loss on iteration 6 230.53568
Current Training Loss on iteration 7 229.3069
Current Training Loss on iteration 8 228.17447
Current Training Loss on iteration 9 227.13081
Current Training Loss on iteration 10 226.16898
Current Training Loss on iteration 11 225.28258
Current Training Loss on iteration 12 224.46564
Current Training Loss on iteration 13 223.71275
Current Training Loss on iteration 14 223.01888
Current Training Loss on iteration 15 222.37941
Current Training Loss on iteration 16 221.79008
Current Training Loss on iteration 17 221.24695
Current Training Loss on iteration 18 220.74641
Current Training Loss on iteration 19 220.28506
Current Training Loss on iteration 20 219.85995
Curren

In [28]:
for i in range(100):    ## 2. observing training loss at every 5th iteration
    W, b = train(X_train_norm, y_train, W, b)
    if(i% 5 == 0) :  ## printing loss at every 5th iteration
      print('Current Training Loss on iteration', i, loss(y_train, prediction(X_train_norm, W, b)).numpy())


Current Training Loss on iteration 0 214.86954
Current Training Loss on iteration 5 214.8671
Current Training Loss on iteration 10 214.86546
Current Training Loss on iteration 15 214.86438
Current Training Loss on iteration 20 214.86368
Current Training Loss on iteration 25 214.8632
Current Training Loss on iteration 30 214.86288
Current Training Loss on iteration 35 214.86267
Current Training Loss on iteration 40 214.86255
Current Training Loss on iteration 45 214.86243
Current Training Loss on iteration 50 214.86237
Current Training Loss on iteration 55 214.86234
Current Training Loss on iteration 60 214.8623
Current Training Loss on iteration 65 214.86229
Current Training Loss on iteration 70 214.86229
Current Training Loss on iteration 75 214.86229
Current Training Loss on iteration 80 214.86227
Current Training Loss on iteration 85 214.86227
Current Training Loss on iteration 90 214.86227
Current Training Loss on iteration 95 214.86227


### Get the shapes and values of W and b

In [29]:
W.numpy()  ## getting values of W weights

array([[1.3151476],
       [1.3194734],
       [1.3014991],
       [1.3306323]], dtype=float32)

In [30]:
print("W shape : ",W.shape)

W shape :  (4, 1)


In [31]:
b.numpy()  ## getting values of bias b

array([2.6335213], dtype=float32)

In [32]:
print("Shape of b Bias ",b.shape)

Shape of b Bias  (1,)


### Model Prediction on 1st Examples in Test Dataset

In [33]:
X_test_norm.shape

(301, 4)

In [0]:
y_pred=prediction(X_test_norm, W, b)

In [35]:
y_pred.shape
y_pred[0:5]

<tf.Tensor: shape=(5, 1), dtype=float32, numpy=
array([[5.266968 ],
       [5.2669716],
       [5.2669573],
       [5.2669616],
       [5.266943 ]], dtype=float32)>

In [36]:
y_test.shape
y_test[0:5]

array([40.6793,  0.5982,  2.1497,  0.819 ,  2.042 ], dtype=float32)

## Classification using tf.Keras

In this exercise, we will build a Deep Neural Network using tf.Keras. We will use Iris Dataset for this exercise.

### Load the given Iris data using pandas (Iris.csv)

In [0]:
iris_df=pd.read_csv('/gdrive/My Drive/ClassNotes/R6_Ext/Iris.csv')

In [38]:
iris_df.tail()

,Sepal Length (in cm),Sepal Width in (cm),Petal length (in cm),Petal width (in cm),Class
145,6.7,3.0,5.2,2.3,Iris-virginica
146,6.3,2.5,5.0,1.9,Iris-virginica
147,6.5,3.0,5.2,2.0,Iris-virginica
148,6.2,3.4,5.4,2.3,Iris-virginica
149,5.9,3.0,5.1,1.8,Iris-virginica


In [39]:
iris_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
Sepal Length (in cm)    150 non-null float64
Sepal Width in (cm)     150 non-null float64
Petal length (in cm)    150 non-null float64
Petal width (in cm)     150 non-null float64
Class                   150 non-null object
dtypes: float64(4), object(1)
memory usage: 6.0+ KB


In [40]:
iris_df.describe()

,Sepal Length (in cm),Sepal Width in (cm),Petal length (in cm),Petal width (in cm)
count,150.000000,150.000000,150.000000,150.000000
mean,5.843333,3.054000,3.758667,1.198667
std,0.828066,0.433594,1.764420,0.763161
min,4.300000,2.000000,1.000000,0.100000
25%,5.100000,2.800000,1.600000,0.300000
50%,5.800000,3.000000,4.350000,1.300000
75%,6.400000,3.300000,5.100000,1.800000
max,7.900000,4.400000,6.900000,2.500000


### Target set has different categories. So, Label encode them. And convert into one-hot vectors using get_dummies in pandas.

In [41]:
iris_df['Class'].unique()

array(['Iris-setosa', 'Iris-versicolor', 'Iris-virginica'], dtype=object)

In [0]:
from sklearn.preprocessing import LabelEncoder
class_label_encoder = LabelEncoder()
iris_df['Class'] = class_label_encoder.fit_transform(iris_df['Class'])


In [43]:
iris_df['Class'].unique()

array([0, 1, 2])

In [0]:
one_hot= pd.get_dummies(iris_df['Class'], prefix='Class')
iris_df = iris_df.drop('Class',axis = 1)
iris_df = iris_df.join(one_hot)



In [45]:
iris_df.head()


,Sepal Length (in cm),Sepal Width in (cm),Petal length (in cm),Petal width (in cm),Class_0,Class_1,Class_2
0,5.1,3.5,1.4,0.2,1,0,0
1,4.9,3.0,1.4,0.2,1,0,0
2,4.7,3.2,1.3,0.2,1,0,0
3,4.6,3.1,1.5,0.2,1,0,0
4,5.0,3.6,1.4,0.2,1,0,0


### Splitting the data into feature set and target set

In [0]:
from scipy.stats import zscore

X = iris_df.drop(labels= ["Class_0","Class_1","Class_2"] , axis = 1)
#X = X.apply(zscore)
y = iris_df[["Class_0","Class_1","Class_2"]]

In [0]:
test_size = 0.30 # taking 70:30 training and test set
seed = 7  # Random numbmer seeding for reapeatability of the code
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=seed)

In [48]:
X_train.shape

(105, 4)

###  Building Model in tf.keras

Build a Linear Classifier model  <br>
1.  Use Dense Layer  with input shape of 4 (according to the feature set) and number of outputs set to 3<br> 
2. Apply Softmax on Dense Layer outputs <br>
3. Use SGD as Optimizer
4. Use categorical_crossentropy as loss function 

In [0]:
# Initialize Sequential model
model = tf.keras.models.Sequential()

# Normalize the data
model.add(tf.keras.layers.BatchNormalization())

# Add Dense Layer which provides 3 Outputs after applying softmax -  activation function would be softmax 
model.add(tf.keras.layers.Dense(3, input_shape=(4,), activation='softmax'))

In [0]:
# Comile the model - Stocashtic Gradient Descent -Stocashtic means random
model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])

### Model Training 

In [0]:
tf.random.set_seed(42)

In [52]:
model.fit(np.array(X_train), np.array(y_train), validation_data=(np.array(X_test), np.array(y_test)), epochs=100, batch_size = y_train.shape[0])

Train on 105 samples, validate on 45 samples
Epoch 1/100
105/105 [==============================] - 0s 3ms/sample - loss: 1.2603 - accuracy: 0.3238 - val_loss: 3.4041 - val_accuracy: 0.2667
Epoch 2/100
105/105 [==============================] - 0s 110us/sample - loss: 1.2395 - accuracy: 0.3238 - val_loss: 3.2958 - val_accuracy: 0.2667
Epoch 3/100
105/105 [==============================] - 0s 94us/sample - loss: 1.2192 - accuracy: 0.3238 - val_loss: 3.1920 - val_accuracy: 0.2667
Epoch 4/100
105/105 [==============================] - 0s 100us/sample - loss: 1.1994 - accuracy: 0.3429 - val_loss: 3.0925 - val_accuracy: 0.2667
Epoch 5/100
105/105 [==============================] - 0s 103us/sample - loss: 1.1802 - accuracy: 0.3429 - val_loss: 2.9971 - val_accuracy: 0.2667
Epoch 6/100
105/105 [==============================] - 0s 102us/sample - loss: 1.1615 - accuracy: 0.3429 - val_loss: 2.9057 - val_accuracy: 0.2667
Epoch 7/100
105/105 [==============================] - 0s 102us/sample - los

After 100 Epochs Accuracy is coming to 82.86 % while validation accuracy is 64.44%..

### Model Prediction

In [0]:
y_pred=model.predict(np.array(X_test))

In [54]:
scores = model.evaluate(np.array(X_train), np.array(y_train), verbose=0)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))


accuracy: 67.62%


In [55]:
scores = model.evaluate(X_test, y_test, verbose=0)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))


accuracy: 64.44%


### Save the Model

In [56]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization (BatchNo multiple                  16        
_________________________________________________________________
dense (Dense)                multiple                  15        
Total params: 31
Trainable params: 23
Non-trainable params: 8
_________________________________________________________________


In [57]:
# Export the model to a SavedModel
model.save('/gdrive/My Drive/ClassNotes/R6_Ext/', save_format='tf')
json_config = model.to_json()


Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: /gdrive/My Drive/ClassNotes/R6_Ext/assets


In [0]:
## retrieving thr model
reinitialized_model = tf.keras.models.model_from_json(json_config)

In [59]:
reinitialized_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization (BatchNo multiple                  16        
_________________________________________________________________
dense (Dense)                multiple                  15        
Total params: 31
Trainable params: 23
Non-trainable params: 8
_________________________________________________________________


### Build and Train a Deep Neural network with 2 hidden layer  - Optional - For Practice

Does it perform better than Linear Classifier? What could be the reason for difference in performance?

In [0]:
# Initialize Sequential model
model2 = tf.keras.models.Sequential()
model2.add(tf.keras.layers.BatchNormalization())
model2.add(tf.keras.layers.Dense(3, input_shape=(4,)))
# Normalize the data
model2.add(tf.keras.layers.BatchNormalization())
# Add Dense Layer which provides 3 Outputs after applying softmax -  activation function would be softmax 
model2.add(tf.keras.layers.Dense(3, input_shape=(4,), activation='softmax'))

# Comile the model - Stocashtic Gradient Descent -Stocashtic means random
model2.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])

In [66]:
model2.fit(np.array(X_train), np.array(y_train), validation_data=(np.array(X_test), np.array(y_test)), epochs=100, batch_size = y_train.shape[0])

Train on 105 samples, validate on 45 samples
Epoch 1/100
105/105 [==============================] - 1s 5ms/sample - loss: 1.4048 - accuracy: 0.2857 - val_loss: 10.5903 - val_accuracy: 0.2667
Epoch 2/100
105/105 [==============================] - 0s 131us/sample - loss: 1.3187 - accuracy: 0.3524 - val_loss: 10.1603 - val_accuracy: 0.2667
Epoch 3/100
105/105 [==============================] - 0s 112us/sample - loss: 1.2568 - accuracy: 0.3810 - val_loss: 9.7695 - val_accuracy: 0.2667
Epoch 4/100
105/105 [==============================] - 0s 104us/sample - loss: 1.2088 - accuracy: 0.4190 - val_loss: 9.4087 - val_accuracy: 0.2667
Epoch 5/100
105/105 [==============================] - 0s 112us/sample - loss: 1.1694 - accuracy: 0.4476 - val_loss: 9.0722 - val_accuracy: 0.2667
Epoch 6/100
105/105 [==============================] - 0s 104us/sample - loss: 1.1357 - accuracy: 0.4762 - val_loss: 8.7560 - val_accuracy: 0.2667
Epoch 7/100
105/105 [==============================] - 0s 105us/sample - 

On increasing the number of dense layer in this case, for 100 epochs, gives accuracy lesser than first , of 88.57, val_accuracy of 64.44%.

In [0]:
y_pred=model2.predict(np.array(X_test))

In [68]:
model2.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_3 (Batch multiple                  16        
_________________________________________________________________
dense_3 (Dense)              multiple                  15        
_________________________________________________________________
batch_normalization_4 (Batch multiple                  12        
_________________________________________________________________
dense_4 (Dense)              multiple                  12        
Total params: 55
Trainable params: 41
Non-trainable params: 14
_________________________________________________________________


But accuracies have been improved from dense count 1 to 2.
82 -> 88 (train) and similar for test though of 64.4%.
Since with increase in layers, the error gradient is supressed, as w go from input to output layer.